In [2]:
#!python -m pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.5/230.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.1/930.1 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.3 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for PyMeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=90c97ed91cd8881345cf22ef9dc7af1785732ff22eb9f12e2014a6500706dd06
  Stored in directory: /opt/ml/.cache/pip/wheels/b9/49/19/1ec4bcf2cc76

In [1]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [2]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director
genre_name_df = pd.read_csv('/opt/ml/input/data/train/genres.tsv', sep='\t') # item-genre(name)
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

## 각 유저별 본 영화의 interaction 수의 통계량(min, max, std)

In [4]:
main_df

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563
...,...,...,...
5154466,138493,44022,1260209449
5154467,138493,4958,1260209482
5154468,138493,68319,1260209720
5154469,138493,40819,1260209726


In [7]:
item_inter_df = pd.read_csv('/opt/ml/input/fighting/FE/item/item_interaction_cnt.csv')

In [9]:
df = main_df.merge(item_inter_df, how='left', on='item')

In [13]:
df

,user,item,time,item_cnt
0,11,4643,1230782529,1122
1,11,170,1230782534,1107
2,11,531,1230782539,966
3,11,616,1230782542,1101
4,11,2140,1230782563,1516
...,...,...,...,...
5154466,138493,44022,1260209449,1282
5154467,138493,4958,1260209482,621
5154468,138493,68319,1260209720,1443
5154469,138493,40819,1260209726,4730


In [11]:
user_grouped = df.groupby('user')

In [15]:
result = user_grouped.agg({'item_cnt':['mean','std']})

In [18]:
result = pd.DataFrame(result)

In [23]:
result.columns = ['user','seen_item_inter_mean','seen_item_inter_std']

In [24]:
result

,user,seen_item_inter_mean,seen_item_inter_std
0,11,3719.351064,4316.497874
1,14,3073.027778,3768.996954
2,18,3769.766234,4812.976459
3,25,7721.142857,4926.058635
4,31,3304.058442,4207.146662
...,...,...,...
31355,138473,7041.571429,5419.559584
31356,138475,1339.217742,2048.243886
31357,138486,4972.875912,4182.405810
31358,138492,5347.294118,4487.995092


In [25]:
result.to_csv('user_seen_item_statistic.csv', index=False)